# **MULTI-FORECASTING VARIABLES 2024–2030**

In [1]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.optimizers import Adam

## **Data Preparation**

### Load Data

In [2]:
df = pd.read_excel("data/gold_1994-2023.xlsx").sort_values("year")

# Target kolom yang HARUS diprediksi
target_cols = [
    "tin",
    "lend_int",
    "Perak",    
    "fed_rate",
    "cpo",
    "real_int",
    "Nickel",
    "ec_growth",
    "exch_rate",
    "inflation",
    "Aluminium"
]

# Validasi kolom tersedia
missing = set(target_cols) - set(df.columns)
if missing:
    raise ValueError(f"Kolom berikut tidak ditemukan di CSV: {missing}")

### Feature Engineering

In [3]:
# Semua kolom kecuali year menjadi fitur
feature_cols = df.columns.drop("year")
values = df[feature_cols].copy()

# Transformasi log
values = np.log1p(values)

# Hitung return YoY (prosentase perubahan)
returns = values.pct_change().dropna().reset_index(drop=True)

# Feature input (semua kolom)
X_data = returns.values

# Target output (hanya target)
y_data = returns[target_cols].values

# Scaling return
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_data)

d:\Projects\Bu Lina\Gold Price Prediction\.venv\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
C:\Users\harry\AppData\Local\Temp\ipykernel_39212\2756255597.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = values.pct_change().dropna().reset_index(drop=True)


### Windowing

In [4]:
# Windowing: gunakan 5 tahun data untuk memprediksi 7 tahun ke depan
X, y = [], []
n_features = X_scaled.shape[1]
n_targets = len(target_cols)

for i in range(len(X_scaled) - 5 - 7 + 1):
    X.append(X_scaled[i:i+5])
    y.append(y_data[i+5:i+12])

X = np.array(X)   # (samples, 5, n_features)
y = np.array(y)   # (samples, 7, n_targets)

# Train/test berdasarkan waktu
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## **Modelling**

In [5]:
model = Sequential([
    Flatten(input_shape=(5, n_features)),
    Dense(512, activation='relu'),
    Dropout(0.1618),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(7 * n_targets, activation='linear'),
    Reshape((7, n_targets))
])

model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=1.618e-2)
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 70)                0         
                                                                 
 dense (Dense)               (None, 512)               36352     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 77)                9933      
                                                                 
 reshape (Reshape)           (None, 7, 11)             0

### Training

In [6]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    verbose=1,
    min_lr=1e-7
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,
    restore_best_weights=True,
    verbose=1
)

callbacks = [reduce_lr, early_stopping]

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,
    batch_size=8,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/300
2/2 [==============================] - 1s 204ms/step - loss: 3.9900 - val_loss: 1.0616 - lr: 0.0162
Epoch 2/300
2/2 [==============================] - 0s 33ms/step - loss: 0.3147 - val_loss: 1.0637 - lr: 0.0162
Epoch 3/300
2/2 [==============================] - 0s 31ms/step - loss: 0.1784 - val_loss: 1.0675 - lr: 0.0162
Epoch 4/300
2/2 [==============================] - 0s 33ms/step - loss: 0.1781 - val_loss: 1.0677 - lr: 0.0162
Epoch 5/300
2/2 [==============================] - 0s 32ms/step - loss: 0.1779 - val_loss: 1.0674 - lr: 0.0162
Epoch 6/300
2/2 [==============================] - 0s 32ms/step - loss: 0.1775 - val_loss: 1.0663 - lr: 0.0162
Epoch 7/300
2/2 [==============================] - 0s 33ms/step - loss: 0.1770 - val_loss: 1.0651 - lr: 0.0162
Epoch 8/300
2/2 [==============================] - 0s 31ms/step - loss: 0.1764 - val_loss: 1.0636 - lr: 0.0162
Epoch 9/300
2/2 [==============================] - 0s 32ms/step - loss: 0.1759 - val_loss: 1.0621 - lr: 0.0162


## **Forecasting 2024-2030**

In [7]:
last_5_scaled = X_scaled[-5:]
pred_scaled = model.predict(np.expand_dims(last_5_scaled, axis=0))[0]   # (7, 11)

# Inverse scaling → tambah dummy kolom agar sesuai jumlah fitur
temp = np.zeros((7, n_features))
temp[:, :n_targets] = pred_scaled
pred_returns_full = scaler.inverse_transform(temp)
pred_returns = pred_returns_full[:, :n_targets]  # (7,11)

# Ubah return → harga absolut
last_real = df[target_cols].iloc[-1].values
forecast_prices = []
cur = last_real.copy()

for i in range(7):
    cur = cur * (1 + pred_returns[i])
    forecast_prices.append(cur)

forecast_df = pd.DataFrame(
    forecast_prices,
    index=[2024, 2025, 2026, 2027, 2028, 2029, 2030],
    columns=target_cols
)

print("\n✔ FORECAST HARGA 2024–2030:")
print(forecast_df)

1/1 [==============================] - 0s 94ms/step

✔ FORECAST HARGA 2024–2030:
               tin  lend_int      Perak  fed_rate        cpo  real_int  \
2024  26164.022272  8.935541  27.690637  4.991042  81.148214  6.811240   
2025  26391.455634  8.996174  27.109353  4.947323  81.532960  6.385658   
2026  26619.148848  9.071082  26.589593  4.916528  81.914478  5.991178   
2027  26849.886889  9.121996  26.020685  4.924095  82.296603  5.616400   
2028  27082.564371  9.211609  25.424848  4.957095  82.680981  5.246770   
2029  27311.680147  9.273435  24.821994  4.997981  83.067370  4.911700   
2030  27539.206202  9.356224  24.207197  5.028623  83.450463  4.698219   

            Nickel  ec_growth     exch_rate  inflation    Aluminium  
2024  20509.477435   5.243755  15224.211400   3.646730  2259.311723  
2025  19539.405649   5.383215  15210.885669   3.627737  2262.862179  
2026  18638.340821   5.530623  15198.199006   3.603150  2266.308531  
2027  17747.796213   5.678363  15186.642337   

## Evaluasi

In [8]:
test_pred_scaled = model.predict(X_test).reshape(-1, n_targets)
test_true = y_test.reshape(-1, n_targets)

# inverse return prediksi
temp2 = np.zeros((len(test_pred_scaled), n_features))
temp2[:, :n_targets] = test_pred_scaled
test_pred_inv = scaler.inverse_transform(temp2)[:, :n_targets]

# inverse return aktual
temp3 = np.zeros((len(test_true), n_features))
temp3[:, :n_targets] = test_true
test_true_inv = scaler.inverse_transform(temp3)[:, :n_targets]

# YoY percentage error
yoy_error_percent = np.mean(np.abs(test_true_inv - test_pred_inv) * 100)

# Directional accuracy
direction_acc = np.mean(
    np.sign(test_true_inv) == np.sign(test_pred_inv)
) * 100

print("\n📌 Evaluasi Model")
print(f"YoY Error (%)              : {yoy_error_percent:.2f}%")
print(f"Directional Accuracy (%)   : {direction_acc:.2f}%")


1/1 [==============================] - 0s 21ms/step

📌 Evaluasi Model
YoY Error (%)              : 2.07%
Directional Accuracy (%)   : 85.39%


## **Gold Regression**

In [9]:
coef = {
    "tin": 544.0605,
    "lend_int": -317.7443,
    "Perak": 302.1824,
    "fed_rate": 205.4201,
    "cpo": 136.2052,
    "real_int": 126.3978,
    "Nickel": -123.0575,
    "ec_growth": -64.8105,
    "exch_rate": -55.5690,
    "inflation": 22.8968,
    "Aluminium": -6.1448
}
intercept = 1007.62354

# menghitung delta / perubahan harga emas (masih dalam 1×Gold units)
gold_delta = []
for i in range(7):
    g = intercept
    for idx, var in enumerate(target_cols):
        g += pred_scaled[i, idx] * coef[var]
    gold_delta.append(g)

# konversi delta → harga absolut kumulatif
gold_price = []
prev_price = df["gold_price"].iloc[-1]

for d in gold_delta:
    new_price = prev_price + d
    gold_price.append(new_price)
    prev_price = new_price

# tambahkan ke forecast_df
forecast_df["gold_price"] = gold_price

print("\n✔ FORECAST FINAL TERMASUK HARGA EMAS (ABSOLUT):")
print(forecast_df)


✔ FORECAST FINAL TERMASUK HARGA EMAS (ABSOLUT):
               tin  lend_int      Perak  fed_rate        cpo  real_int  \
2024  26164.022272  8.935541  27.690637  4.991042  81.148214  6.811240   
2025  26391.455634  8.996174  27.109353  4.947323  81.532960  6.385658   
2026  26619.148848  9.071082  26.589593  4.916528  81.914478  5.991178   
2027  26849.886889  9.121996  26.020685  4.924095  82.296603  5.616400   
2028  27082.564371  9.211609  25.424848  4.957095  82.680981  5.246770   
2029  27311.680147  9.273435  24.821994  4.997981  83.067370  4.911700   
2030  27539.206202  9.356224  24.207197  5.028623  83.450463  4.698219   

            Nickel  ec_growth     exch_rate  inflation    Aluminium  \
2024  20509.477435   5.243755  15224.211400   3.646730  2259.311723   
2025  19539.405649   5.383215  15210.885669   3.627737  2262.862179   
2026  18638.340821   5.530623  15198.199006   3.603150  2266.308531   
2027  17747.796213   5.678363  15186.642337   3.576764  2269.570797   
202